In [227]:
# Importando bibliotecas
from unidecode import unidecode
from urllib.request import urlopen, urlretrieve, Request
from bs4 import BeautifulSoup
import pandas as pd
import math
import requests
from urllib.error import HTTPError


def dados_site(site): # Função abrir o site e carregar no beautifulsoup

    headers = {'User-Agent': 'Mozilla/5.0'}

    try:
        req = Request(site, headers = headers)
        response = urlopen(req)
        
    except HTTPError as e:
        print(e.status, e.reason)
        return "404 Not Found"
        
        
    except URLError as e:
        print(e.reason)
        return "404 Not Found"

    html = response.read().decode('utf-8')
    html_soup = BeautifulSoup(html, 'html.parser')
    return html_soup



url = 'https://www.otvfoco.com.br/audiencias-detalhadas/' #Site onde vamos fazer a raspagem de dados
site_de_novelas = dados_site(url) #Chamamos a função para abrir e retorna o site


In [94]:
novelas = []
#Aqui vamos captar pagina por pagina, acessar e extrair o que queremos
for novela_extraida in site_de_novelas.find('div',{"class":"entry-content"}).findAll("a"):
    novela = {}
    novela_site = dados_site(novela_extraida.get('href')) #Pegamos os link de todos os sites
    texto = novela_extraida.text #Pega o texto da novela
    texto = texto.replace('/','–')#Troca \ por – para que possa ser fatiado com mais precisão
    texto = texto.split('–') 
    if len(texto)<2:
        continue
    novela['imagem'] = novela_site.find('figure').find('img').get('data-src')
    novela['Audiencia Média'] = novela_site.find('div',{"class":"col-lg-8 col-md-12"}).findAll('p')[-2].text
    novela['Audiencia Média'] = novela['Audiencia Média'].replace('MÉDIA GERAL: ','')
    novela['Nome'] = texto[0]
    novela['Ano'] = texto[-3]
    novela['Autor'] = texto[-1]
    novelas.append(novela) # Adicionar a novela a lista de novelas


CORAÇÃO ALADO 
BAILA COMIGO 
BRILHANTE 
SÉTIMO SENTIDO 
SOL DE VERÃO 
LOUCO AMOR 
CHAMPAGNE 
PARTIDO ALTO 
CORPO A CORPO 
ROQUE SANTEIRO 
SELVA DE PEDRA 
RODA DE FOGO 
O OUTRO 
MANDALA 
VALE TUDO 
O SALVADOR DA PÁTRIA 
TIETA 
RAINHA DA SUCATA 
MEU BEM, MEU MAL 
O DONO DO MUNDO 
PEDRA SOBRE PEDRA 
DE CORPO E ALMA 
RENASCER 
FERA FERIDA 
PÁTRIA MINHA 
A PRÓXIMA VÍTIMA 
EXPLODE CORAÇÃO 
O FIM DO MUNDO 
O REI DO GADO 
A INDOMADA 
POR AMOR 
TORRE DE BABEL 
SUAVE VENENO 
TERRA NOSTRA 
LAÇOS DE FAMÍLIA 
PORTO DOS MILAGRES 
O CLONE 
ESPERANÇA 
MULHERES APAIXONADAS 
CELEBRIDADE 
SENHORA DO DESTINO 
AMÉRICA 
BELÍSSIMA 
PÁGINAS DA VIDA 
PARAÍSO TROPICAL 
DUAS CARAS 
A FAVORITA 
CAMINHO DAS ÍNDIAS 
VIVER A VIDA 
PASSIONE 
INSENSATO CORAÇÃO 
FINA ESTAMPA 
AVENIDA BRASIL 
SALVE JORGE 
AMOR À VIDA 
EM FAMÍLIA 
IMPÉRIO 
BABILÔNIA 
A REGRA DO JOGO 
VELHO CHICO 
A LEI DO AMOR 
A FORÇA DO QUERER 
O OUTRO LADO DO PARAÍSO 
SEGUNDO SOL 
O SÉTIMO GUARDIÃO 
A DONA DO PEDAÇO 
AMOR DE MÃE 
FINA ESTAMPA (edição 

In [260]:
df_Novelas = pd.DataFrame(novelas)
df_Novelas

,imagem,Audiencia Média,Nome,Ano,Autor
0,https://stcotvfoco.com.br/2020/11/coracaoalado...,56.88,CORAÇÃO ALADO,1980,Janete Clair
1,https://stcotvfoco.com.br/2020/11/bailacomigo.jpg,55.51,BAILA COMIGO,1981,Manoel Carlos
2,https://stcotvfoco.com.br/2020/11/brilhante.png,44.64,BRILHANTE,1981,Gilberto Braga
3,https://stcotvfoco.com.br/2020/11/setimosentid...,45.96,SÉTIMO SENTIDO,1982,Janete Clair
4,https://stcotvfoco.com.br/2020/11/soldeverao.jpg,44.80,SOL DE VERÃO,1982,Manoel Carlos
...,...,...,...,...,...
424,https://stcotvfoco.com.br/2020/12/floribella.jpg,3.78,FLORIBELLA 1ª TEMP,2005,Patrícia Moretzsohn e Jaqueline Vargas
425,https://stcotvfoco.com.br/2020/12/floribella.jpg,MÉDIA PARCIAL: 2.98,FLORIBELLA 2ª TEMP,2006,Patrícia Moretzsohn e Jaqueline Vargas
426,https://stcotvfoco.com.br/2020/12/paixoes.jpg,2.07,PAIXÕES PROIBIDAS,2006,Aimar Labaki
427,https://stcotvfoco.com.br/2020/12/dancedanceda...,2.97,DANCE DANCE DANCE,2007,Juana Uribe e Yoya Wursch


In [232]:
def normalizar_nome(texto):
    texto = texto.replace(' ', '-')
    texto = texto.lower()[:-1]
    texto = unidecode(texto)
    return texto

def achar_Tema_de_abertura(lista_site):

    for linha in lista_site:
        if linha.text.find('Tema de Abertura:')!= -1:
            musica_abertura = linha.text
            musica_abertura = musica_abertura.replace('Tema de Abertura: ','')
            return musica_abertura
    
def canal_e_horario(op_hr):
    op_hr = op_hr.split('\n')
    op_hr = op_hr[0]
    op_hr = op_hr.split('–')
    x = op_hr[0]
    y = op_hr[-1]
    return x,y

dados_novelas = []


### 
# Como o primeiro site não tinha todas as informações que eu gostaria no dataset, achei as informações em um outro site onde vamos
#fazer a raspagem para adicionar tambem ao nosso dataset
# ###

for novela_extraida in novelas:
    dado_novela = {}
    nome_novela = novela_extraida['Nome']
    nome_novela = normalizar_nome(nome_novela)
    url = 'http://teledramaturgia.com.br/' + nome_novela

    print(url)
    dados_novela = dados_site(url)
    if dados_novela == '404 Not Found':
        continue
    try:
        dado_novela['Nome'] = novela_extraida['Nome']
        dado_novela['Site'] = url
        dado_novela['Sinopse'] = dados_novela.find('div',{"class":"fusion-column-wrapper"}).text
        dado_novela['Elenco'] = dados_novela.find('div',{"class":"tab-content"}).findAll('div')[0].text
        dado_novela['Abertura'] = achar_Tema_de_abertura(dados_novela.find('div',{"class":"tab-content"}).findAll('p'))
        dado_novela['Canal'],dado_novela['Horario'] = canal_e_horario(dados_novela.find('div',{"class":"content-container"}).findAll('p')[0].text)
    except:
        pass
    dados_novelas.append(dado_novela)  
    

    

    


http://teledramaturgia.com.br/coracao-alado
http://teledramaturgia.com.br/baila-comigo
http://teledramaturgia.com.br/brilhante
http://teledramaturgia.com.br/setimo-sentido
http://teledramaturgia.com.br/sol-de-verao
http://teledramaturgia.com.br/louco-amor
http://teledramaturgia.com.br/champagne
http://teledramaturgia.com.br/partido-alto
http://teledramaturgia.com.br/corpo-a-corpo
http://teledramaturgia.com.br/roque-santeiro
http://teledramaturgia.com.br/selva-de-pedra
http://teledramaturgia.com.br/roda-de-fogo
http://teledramaturgia.com.br/o-outro
http://teledramaturgia.com.br/mandala
http://teledramaturgia.com.br/vale-tudo
http://teledramaturgia.com.br/o-salvador-da-patria
http://teledramaturgia.com.br/tieta
http://teledramaturgia.com.br/rainha-da-sucata
http://teledramaturgia.com.br/meu-bem,-meu-mal
http://teledramaturgia.com.br/o-dono-do-mundo
http://teledramaturgia.com.br/pedra-sobre-pedra
http://teledramaturgia.com.br/de-corpo-e-alma
http://teledramaturgia.com.br/renascer
http://t

In [261]:
df_detalhes = pd.DataFrame(dados_novelas)
df_detalhes

,Nome,Site,Sinopse,Elenco,Abertura,Canal,Horario
0,CORAÇÃO ALADO,http://teledramaturgia.com.br/coracao-alado,O escultor pernambucano Juca Pitanga enviava s...,\n\nTARCÍSIO MEIRA – Juca Pitanga\nVERA FISCHE...,NOTURNO – Fagner,Globo,20h
1,BAILA COMIGO,http://teledramaturgia.com.br/baila-comigo,"No passado, Helena, uma adolescente de origem ...",\n\nTONY RAMOS – Quinzinho (Joaquim Seixas Mir...,BAILA COMIGO – Robson Jorge e Lincoln Olivetti...,Globo,20h
2,BRILHANTE,http://teledramaturgia.com.br/brilhante,Luiza é designer de uma empresa de jóias. De f...,\n\nVERA FISCHER – LuizaTARCÍSIO MEIRA – Paulo...,LUIZA – Tom Jobim,Globo,20h
3,SÉTIMO SENTIDO,http://teledramaturgia.com.br/setimo-sentido,A marroquina Luana Camará volta ao Brasil para...,\n\nREGINA DUARTE – Luana Camará / Priscila Ca...,AS VITRINES – Chico Buarque,Globo,20h
4,SOL DE VERÃO,http://teledramaturgia.com.br/sol-de-verao,"Infeliz no casamento, Raquel decide se separar...",\n\nJARDEL FILHO – HeitorIRENE RAVACHE – Raque...,TÔ QUE TÔ – Simone,Globo,20h
...,...,...,...,...,...,...,...
329,AMOR SEM IGUAL,http://teledramaturgia.com.br/amor-sem-igual,A prostituta Angélica – de codinome Poderosa (...,\nDAY MESQUITA – Angélica / PoderosaRAFAEL SAR...,None,Record,55 capítulostotal: 148 capítulos
330,GÊNESIS,http://teledramaturgia.com.br/genesis,"Os 2.300 primeiros anos da Humanidade, com a o...",\n\nJardim do Éden – de 19 a 21/01/2021FLÁVIO ...,COMEÇO DE TUDO – Banda Universos,Record,21hde 19 de janeiro a 22 de novembro de 20212...
331,PAIXÕES PROIBIDAS,http://teledramaturgia.com.br/paixoes-proibidas,Três histórias de amor têm como cenário os con...,\n\nMIGUEL THIRÉ – Simão de AzevedoANA SOPHIA ...,ROMANCE DA MORENINHA – Alceu Valença *,Band,22h / 17h30de 14 de novembro de 2006a 8 de ju...
332,DANCE DANCE DANCE,http://teledramaturgia.com.br/dance-dance-dance,Sofia Ivanitch herdou da avó o nome e o talent...,\n\nJULIANA BARONI – SofiaRICARDO MARTINS – Ra...,DANCE DANCE DANCE,Band,20h15


In [262]:
#Vamos fazer um join para unir os dois dataframes em um unico dataframe final
dataset_final = pd.merge(df_Novelas, df_detalhes, on= ['Nome'], how="left")
dataset_final

,imagem,Audiencia Média,Nome,Ano,Autor,Site,Sinopse,Elenco,Abertura,Canal,Horario
0,https://stcotvfoco.com.br/2020/11/coracaoalado...,56.88,CORAÇÃO ALADO,1980,Janete Clair,http://teledramaturgia.com.br/coracao-alado,O escultor pernambucano Juca Pitanga enviava s...,\n\nTARCÍSIO MEIRA – Juca Pitanga\nVERA FISCHE...,NOTURNO – Fagner,Globo,20h
1,https://stcotvfoco.com.br/2020/11/bailacomigo.jpg,55.51,BAILA COMIGO,1981,Manoel Carlos,http://teledramaturgia.com.br/baila-comigo,"No passado, Helena, uma adolescente de origem ...",\n\nTONY RAMOS – Quinzinho (Joaquim Seixas Mir...,BAILA COMIGO – Robson Jorge e Lincoln Olivetti...,Globo,20h
2,https://stcotvfoco.com.br/2020/11/brilhante.png,44.64,BRILHANTE,1981,Gilberto Braga,http://teledramaturgia.com.br/brilhante,Luiza é designer de uma empresa de jóias. De f...,\n\nVERA FISCHER – LuizaTARCÍSIO MEIRA – Paulo...,LUIZA – Tom Jobim,Globo,20h
3,https://stcotvfoco.com.br/2020/11/setimosentid...,45.96,SÉTIMO SENTIDO,1982,Janete Clair,http://teledramaturgia.com.br/setimo-sentido,A marroquina Luana Camará volta ao Brasil para...,\n\nREGINA DUARTE – Luana Camará / Priscila Ca...,AS VITRINES – Chico Buarque,Globo,20h
4,https://stcotvfoco.com.br/2020/11/soldeverao.jpg,44.80,SOL DE VERÃO,1982,Manoel Carlos,http://teledramaturgia.com.br/sol-de-verao,"Infeliz no casamento, Raquel decide se separar...",\n\nJARDEL FILHO – HeitorIRENE RAVACHE – Raque...,TÔ QUE TÔ – Simone,Globo,20h
...,...,...,...,...,...,...,...,...,...,...,...
722,https://stcotvfoco.com.br/2020/12/floribella.jpg,3.78,FLORIBELLA 1ª TEMP,2005,Patrícia Moretzsohn e Jaqueline Vargas,NaN,NaN,NaN,NaN,NaN,NaN
723,https://stcotvfoco.com.br/2020/12/floribella.jpg,MÉDIA PARCIAL: 2.98,FLORIBELLA 2ª TEMP,2006,Patrícia Moretzsohn e Jaqueline Vargas,NaN,NaN,NaN,NaN,NaN,NaN
724,https://stcotvfoco.com.br/2020/12/paixoes.jpg,2.07,PAIXÕES PROIBIDAS,2006,Aimar Labaki,http://teledramaturgia.com.br/paixoes-proibidas,Três histórias de amor têm como cenário os con...,\n\nMIGUEL THIRÉ – Simão de AzevedoANA SOPHIA ...,ROMANCE DA MORENINHA – Alceu Valença *,Band,22h / 17h30de 14 de novembro de 2006a 8 de ju...
725,https://stcotvfoco.com.br/2020/12/dancedanceda...,2.97,DANCE DANCE DANCE,2007,Juana Uribe e Yoya Wursch,http://teledramaturgia.com.br/dance-dance-dance,Sofia Ivanitch herdou da avó o nome e o talent...,\n\nJULIANA BARONI – SofiaRICARDO MARTINS – Ra...,DANCE DANCE DANCE,Band,20h15


In [256]:
def remover_n (elenco):
    elenco = str(elenco)
    
    return elenco.replace('\n', '')

def limpar_hora (hora):
    hora = str(hora)
     
    return hora[:4]
### 
#Vamos Limpar o elenco que aparece alguns \n e o horario que para somentar apaerecer a primeira hora
###
dataset_final['Elenco'] = dataset_final['Elenco'].apply(remover_n)
dataset_final['Horario'] = dataset_final['Horario'].apply(limpar_hora)


In [271]:
dataset_final = dataset_final.drop_duplicates()
dataset_final.to_csv('novelas.csv')